# Tourism #
We are interested in producing hierarchical reconciled forecasts for the Tourism dataset.

This example is based on chapter 11 of:

Hyndman, R.J., & Athanasopoulos, G. (2021) *Forecasting: principles and practice, 
3rd edition*, OTexts: Melbourne, Australia. OTexts.com/fpp3. Accessed on 05/07/2022.

The data was taken from R's tsibble package:

Wang, E, D Cook, and RJ Hyndman (2020). *A new tidy data structure to support exploration 
and modeling of temporal data*, Journal of Computational and Graphical Statistics, 29:3, 
466-478, doi:10.1080/10618600.2019.1695624.

https://tsibble.tidyverts.org/

Download the tourism.csv file at: https://github.com/elephaint/hierts/tree/main/examples/data

The code for this example can be found at:

* Jupyter Notebook: https://github.com/elephaint/hierts/tree/main/docs/notebooks
* Python script: https://github.com/elephaint/hierts/tree/main/examples

### Import packages & read data

In [4]:
#%% Read packages
import pandas as pd
import numpy as np
from hierts.reconciliation import calc_summing_matrix, apply_reconciliation_methods, \
                                  aggregate_bottom_up_forecasts, calc_level_method_rmse

In [5]:
#%% Read data
df = pd.read_csv('../../examples/data/tourism.csv', index_col=0)
# We need to convert the Quarter column to a PeriodIndex in order to guarantee correct order
df['Quarter'] = pd.PeriodIndex(df['Quarter'].str[0:4] + '-' + df['Quarter'].str[5:], freq='q')

In [6]:
# Let's look at the data
df.head(10)

,Quarter,Region,State,Purpose,Trips
1,1998Q1,Adelaide,SA,Business,135.077690
2,1998Q2,Adelaide,SA,Business,109.987316
3,1998Q3,Adelaide,SA,Business,166.034687
4,1998Q4,Adelaide,SA,Business,127.160464
5,1999Q1,Adelaide,SA,Business,137.448533
6,1999Q2,Adelaide,SA,Business,199.912586
7,1999Q3,Adelaide,SA,Business,169.355090
8,1999Q4,Adelaide,SA,Business,134.357937
9,2000Q1,Adelaide,SA,Business,154.034398
10,2000Q2,Adelaide,SA,Business,168.776364


As you can see, we have a column indicating the time index ('Quarter'), three columns that provide information on the hierarchy/aggregation of the data ('State', 'Region', 'Purpose'), and a target column ('Trips').

### Set aggregations and calculate summing matrix
For this dataset, the aggregations are in the columns: ['State', 'Region', 'Purpose']. These columns will be in our aggregations.

In [7]:
aggregation_cols = ['State', 'Region', 'Purpose']

Next, we define the aggregations that we are interested in. In this case, we are interested in the following aggregations:

In [8]:
aggregations = [['State'],
                ['State', 'Region'],
                ['State', 'Purpose'],
                ['Purpose']]

Don't include the top (total) level and bottom-level: these will be added automatically later on. Now, we can create a summing matrix, which shows how each of the bottom-level series maps to our chosen aggregations:

In [9]:
df_S = calc_summing_matrix(df, aggregation_cols, aggregations)

Let's have a look at df_S:

In [10]:
df_S

bottom_timeseries                                     ACT-Canberra-Business  \
Aggregation       Value                                                       
Total             Total                                                 1.0   
State             ACT                                                   1.0   
                  NSW                                                   0.0   
                  NT                                                    0.0   
                  QLD                                                   0.0   
...                                                                     ...   
bottom_timeseries WA-Australia's South West-Visiting                    0.0   
                  WA-Experience Perth-Business                          0.0   
                  WA-Experience Perth-Holiday                           0.0   
                  WA-Experience Perth-Other                             0.0   
                  WA-Experience Perth-Visiting                          0.0   

bottom_timeseries                                     ACT-Canberra-Holiday  \
Aggregation       Value                                                      
Total             Total                                                1.0   
State             ACT                                                  1.0   
                  NSW                                                  0.0   
                  NT                                                   0.0   
                  QLD                                                  0.0   
...                                                                    ...   
bottom_timeseries WA-Australia's South West-Visiting                   0.0   
                  WA-Experience Perth-Business                         0.0   
                  WA-Experience Perth-Holiday                          0.0   
                  WA-Experience Perth-Other                            0.0   
                  WA-Experience Perth-Visiting                         0.0   

bottom_timeseries                                     ACT-Canberra-Other  \
Aggregation       Value                                                    
Total             Total                                              1.0   
State             ACT                                                1.0   
                  NSW                                                0.0   
                  NT                                                 0.0   
                  QLD                                                0.0   
...                                                                  ...   
bottom_timeseries WA-Australia's South West-Visiting                 0.0   
                  WA-Experience Perth-Business                       0.0   
                  WA-Experience Perth-Holiday                        0.0   
                  WA-Experience Perth-Other                          0.0   
                  WA-Experience Perth-Visiting                       0.0   

bottom_timeseries                                     ACT-Canberra-Visiting  \
Aggregation       Value                                                       
Total             Total                                                 1.0   
State             ACT                                                   1.0   
                  NSW                                                   0.0   
                  NT                                                    0.0   
                  QLD                                                   0.0   
...                                                                     ...   
bottom_timeseries WA-Australia's South West-Visiting                    0.0   
                  WA-Experience Perth-Business                          0.0   
                  WA-Experience Perth-Holiday                           0.0   
                  WA-Experience Perth-Other                             0.0   
                  WA

As you can see, df_S is a mapping between the bottom-level series (columns of df_S) and the aggregations specified. Some observations:

* 'Total' contains the total across all bottom-level series. Hence, this is a row consisting of ones - the total is the sum of all the bottom-level series.
* 'Bottom level' contains the bottom level series. This is an identity matrix, as each bottom-level series in the rows only maps to a single bottom level series in the columns
* Everything in between 'Bottom level' and 'Total' denotes how we can construct the aggregations. For example, the 'State-ACT' aggregation is formed by summing the first four bottom-level series ACT-Canberra-Business, ACT-Canberra-Holiday, ACT-Canberra-Other and ACT-Canberra-Visiting.

### Create a forecasting model for each time series
Now we can create a forecasting model for each time series in the aggregation matrix df_S

In [11]:
# Set target, time_index and split of train and test.
target = 'Trips'
time_index = 'Quarter'
end_train = '2015Q4'
start_test = '2016Q1'

In [12]:
# Add bottom_timeseries identifier and create actuals dataframe for all aggregations
df['bottom_timeseries'] = df[aggregation_cols].agg('-'.join, axis=1)
actuals_bottom_timeseries = df.set_index(['bottom_timeseries', time_index])[target]\
                              .unstack(1)\
                              .loc[df_S.columns]
actuals = df_S @ actuals_bottom_timeseries

Let's look at the actuals:

In [13]:
actuals

Quarter                                                     1998Q1  \
Aggregation       Value                                              
Total             Total                               23182.197269   
State             ACT                                   551.001921   
                  NSW                                  8039.794795   
                  NT                                    181.448823   
                  QLD                                  4041.370159   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    116.970746   
                  WA-Experience Perth-Business          187.985188   
                  WA-Experience Perth-Holiday           294.678011   
                  WA-Experience Perth-Other              42.414022   
                  WA-Experience Perth-Visiting          226.134737   

Quarter                                                     1998Q2  \
Aggregation       Value                                              
Total             Total                               20323.380067   
State             ACT                                   416.025623   
                  NSW                                  7166.013805   
                  NT                                    313.936152   
                  QLD                                  3967.904653   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    106.044650   
                  WA-Experience Perth-Business          118.678907   
                  WA-Experience Perth-Holiday           277.422409   
                  WA-Experience Perth-Other              35.587768   
                  WA-Experience Perth-Visiting          237.021622   

Quarter                                                     1998Q3  \
Aggregation       Value                                              
Total             Total                               19826.640511   
State             ACT                                   436.029011   
                  NSW                                  6747.935790   
                  NT                                    528.436859   
                  QLD                                  4593.893991   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    102.641667   
                  WA-Experience Perth-Business          129.135987   
                  WA-Experience Perth-Holiday           250.539302   
                  WA-Experience Perth-Other              48.137064   
                  WA-Experience Perth-Visiting          235.038378   

Quarter                                                     1998Q4  \
Aggregation       Value                                              
Total             Total                               20830.129891   
State             ACT                                   449.798445   
                  NSW                                  7282.082371   
                  NT                                    247.702817   
                  QLD                                  4202.829141   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    155.877388   
                  WA-Experience Perth-Business          188.048404   
                  WA-Experience Perth-Holiday           291.984508   
                  WA-Experience Perth-Other              61.052428   
                  WA-Experience Perth-Visiting          291.621175   

Quarter                                                     1999Q1  \
Aggregation       Value                                              
Total             Total                               22087.353380   
State             ACT                                   378.572817   
                  NSW                                  7584.776839   
      

As can be seen, by taking the dot product of our summing matrix df_S with our bottom-level actuals, we have obtained a matrix where the aggregations are the rows and the timesteps the columns.

We can now proceed to create forecasts.

In [14]:
# Create forecasts: a simple ETS model per timeseries
from sktime.forecasting.ets import AutoETS
forecasts = pd.DataFrame(index=actuals.index, columns = actuals.columns, dtype=np.float32)
for index, series in actuals.iterrows():
    # Fit model and predict (we need to clip because otherwise there's a convergence error)
    model = AutoETS(auto=True, n_jobs=1, random_state=0)
    model.fit(np.clip(series.loc[:end_train], 1e-3, 1e16))
    forecast = model.predict(series.index)
    # Store to forecasts/actuals array
    forecasts.loc[index] = forecast.values

c:\Users\ospra\miniconda3\envs\phd\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Let's look at the forecasts:

In [15]:
forecasts

Quarter                                                     1998Q1  \
Aggregation       Value                                              
Total             Total                               21574.027344   
State             ACT                                   494.419067   
                  NSW                                  7604.467773   
                  NT                                    345.485168   
                  QLD                                  4156.833496   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    117.684669   
                  WA-Experience Perth-Business          154.717407   
                  WA-Experience Perth-Holiday           283.353638   
                  WA-Experience Perth-Other              46.703793   
                  WA-Experience Perth-Visiting          231.525131   

Quarter                                                     1998Q2  \
Aggregation       Value                                              
Total             Total                               22083.279297   
State             ACT                                   500.439117   
                  NSW                                  7721.035645   
                  NT                                    345.468750   
                  QLD                                  4131.653809   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    117.533379   
                  WA-Experience Perth-Business          160.191299   
                  WA-Experience Perth-Holiday           285.672760   
                  WA-Experience Perth-Other              45.286228   
                  WA-Experience Perth-Visiting          229.379745   

Quarter                                                     1998Q3  \
Aggregation       Value                                              
Total             Total                               21525.980469   
State             ACT                                   501.933624   
                  NSW                                  7572.416992   
                  NT                                    345.465607   
                  QLD                                  4095.944336   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    115.098701   
                  WA-Experience Perth-Business          153.360840   
                  WA-Experience Perth-Holiday           283.983185   
                  WA-Experience Perth-Other              42.081352   
                  WA-Experience Perth-Visiting          232.421234   

Quarter                                                     1998Q4  \
Aggregation       Value                                              
Total             Total                               20987.857422   
State             ACT                                   502.476624   
                  NSW                                  7351.645020   
                  NT                                    345.483917   
                  QLD                                  4204.534668   
...                                                            ...   
bottom_timeseries WA-Australia's South West-Visiting    112.458824   
                  WA-Experience Perth-Business          149.374878   
                  WA-Experience Perth-Holiday           277.134186   
                  WA-Experience Perth-Other              44.082474   
                  WA-Experience Perth-Visiting          233.462860   

Quarter                                                     1999Q1  \
Aggregation       Value                                              
Total             Total                               20937.910156   
State             ACT                                   502.227631   
                  NSW                                  7333.018066   
      

Observations:

* The AutoETS in this case isn't very powerful: it has mostly created constant forecasts for all timesteps in our test set (see after 2015Q4).
* We create forecasts for both the train and test set. This is because we need the residuals (forecast errors) on our training set.

### Reconciliation
We can now reconcile the forecasts. 

In [16]:
# Create forecast test set and apply a set of reconciliation methods.
forecasts_test = forecasts.loc[:, start_test:]
forecasts_methods = apply_reconciliation_methods(forecasts_test, df_S, 
                                                actuals.loc[:, :end_train], forecasts.loc[:, :end_train],
                                                methods=['ols', 'wls_struct', 'wls_var', 'mint_shrink'])

Method ols, reconciliation time: 0.0021s
Method wls_struct, reconciliation time: 0.0013s
Method wls_var, reconciliation time: 0.0012s
Method mint_shrink, reconciliation time: 4.1298s


We will compare our reconciled forecasts to two cases: (i) the unreconciled forecasts ('base'), and (ii) the bottom-up forecasts ('bottom-up'). The latter is obtained by merely summing our bottom-level forecasts for all the aggregations (i.e. we effectively dispense with the separate forecasts for all the higher-level aggregations).

In [19]:
# Create the bottom-up forecasts and add them to the forecasts_methods dataframe
forecasts_bu_bottom_level = forecasts.loc['bottom_timeseries']
forecasts_bu = aggregate_bottom_up_forecasts(forecasts_bu_bottom_level, df_S)
forecasts_bu_test = forecasts_bu.loc[:, start_test:]
forecasts_method = pd.concat({'bottom-up': forecasts_bu_test}, names=['Method'])
forecasts_methods = pd.concat((forecasts_method, forecasts_methods), axis=0)

Finally, we can compute the root mean-squared error on all of our methods.

In [20]:
# Calculate error for all levels and methods. We set bottom-up as the base method to compare against
# in the relative rmse
rmse, rel_rmse = calc_level_method_rmse(forecasts_methods, actuals, base='base')

Let's look at the errors:

In [21]:
rmse

Method,bottom-up,base,ols,wls_struct,wls_var,mint_shrink
Aggregation,,,,,,
Total,3306.986700,2384.445914,2404.999427,2738.143230,2927.180695,1755.004329
Purpose,1087.956677,1070.998394,1060.990947,1029.587893,1025.144238,791.360905
State,654.361105,567.977400,558.927221,591.119925,614.470862,464.545683
State-Purpose,221.070602,219.723817,218.963422,215.304224,215.310404,183.111974
State-Region,93.707037,94.867123,87.337924,88.549785,87.412677,77.612773
bottom_timeseries,36.514784,36.514784,36.118552,35.815110,35.462472,33.346821
All series,226.133054,190.903411,189.715393,200.013370,207.160978,146.971670


We can see the following:

* The base forecast, where we create a separate forecast for each time series in each aggregation, has an overall RMSE of 191.
* The base forecast has been reconciled using 'ols', 'wls_struct', 'wls_var' and 'mint_shrink'. Of these 4 methods, 'mint_shrink' achieves superior performance; across all levels the forecast is better than the base forecast.
* The bottom-up forecast performs rather poorly. Note that (by definition) base == bottom-up for the Bottom level aggregation.

Finally, let's look at the relative rmse. This provides an easier insight into which method performs best. We compare against the base forecast.

In [22]:
rel_rmse

Method,bottom-up,base,ols,wls_struct,wls_var,mint_shrink
Aggregation,,,,,,
Total,1.386899,1.0,1.008620,1.148335,1.227615,0.736022
Purpose,1.015834,1.0,0.990656,0.961335,0.957186,0.738900
State,1.152090,1.0,0.984066,1.040746,1.081858,0.817895
State-Purpose,1.006129,1.0,0.996539,0.979886,0.979914,0.833373
State-Region,0.987771,1.0,0.920634,0.933409,0.921422,0.818121
bottom_timeseries,1.000000,1.0,0.989149,0.980839,0.971181,0.913242
All series,1.184542,1.0,0.993777,1.047720,1.085161,0.769875


Clearly, 'mint_shrink' provides the best performance, across all series it performs about 25% better than the base forecast.